## Consigne

In [ ]:
# Importation des librairies classiques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Importation du fichier CSV
# Assure-toi que le chemin est correct. Si ton notebook est dans 'notebooks'
# et tes données dans 'data/raw', il faut remonter d'un cran avec '..'
try:
    df = pd.read_csv('../data/raw/insurance.csv')
    print("✅ Fichier chargé avec succès !")
except FileNotFoundError:
    print("❌ Erreur : Fichier introuvable. Vérifie le chemin ou le nom du fichier.")

# 2. Vérification de la structure et des valeurs manquantes
print("\n--- Infos sur le DataFrame ---")
df.info()

print("\n--- Vérification des valeurs manquantes (NaN) ---")
print(df.isna().sum())

# 3. Statistiques descriptives de base
print("\n--- Statistiques Descriptives   ---")
display(df.describe()) # 'display()' rend le tableau plus joli dans Jupyter

## Étude de Cas A : Le BMI dépend-il de la région ? (ANOVA)

##### Question 1 : Analyse Visuelle (Validation intuitive)

In [ ]:
# Étude de Cas A - Question 1 : Analyse Visuelle
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# On trace le Boxplot
# x = Variable Qualitative (Région)
# y = Variable Quantitative (BMI)
sns.boxplot(x='region', y='bmi', data=df, palette='Set2')

plt.title('Distribution du BMI par Région', fontsize=16)
plt.xlabel('Région', fontsize=12)
plt.ylabel('BMI (Indice de Masse Corporelle)', fontsize=12)

plt.show()


- Différence de moyenne : On remarque que la région Southeast semble avoir un BMI médian plus élevé que les trois autres régions.

- Stabilité de la variance : Les boîtes (hauteur interquartile) ont des tailles très comparables. Cela suggère que la variabilité (variance) est la même dans chaque groupe.

- La condition d'homogénéité des variances semble respectée visuellement. On peut donc procéder au test ANOVA.

##### Question 2 : Statistiques Descriptives

In [ ]:
# Question 2 : Statistiques Descriptives
# On groupe par 'region', on prend la colonne 'bmi', et on calcule mean & std
stats_bmi = df.groupby('region')['bmi'].agg(['mean', 'std'])

# On arrondit à 2 chiffres après la virgule pour que ce soit propre
print(stats_bmi.round(2))

Les écarts-types sont très proches. Ils varient légèrement entre 5.14 (Northwest) et 6.48 (Southeast), gravitant tous autour de la valeur 6.0. Cela confirme que la dispersion des données est similaire entre les groupes, ce qui valide l'hypothèse d'homogénéité des variances nécessaire pour effectuer l'ANOVA.

##### Question 3 : Test ANOVA

In [ ]:
# Question 3 : Test ANOVA (One-Way)
from scipy.stats import f_oneway

# 1. On prépare les 4 équipes (les 4 groupes de BMI)
bmi_sw = df[df['region'] == 'southwest']['bmi']
bmi_se = df[df['region'] == 'southeast']['bmi']
bmi_nw = df[df['region'] == 'northwest']['bmi']
bmi_ne = df[df['region'] == 'northeast']['bmi']

# 2. On lance le test ANOVA
# f_stat = La puissance du test
# p_value = Le juge (Verdict)
f_stat, p_value = f_oneway(bmi_sw, bmi_se, bmi_nw, bmi_ne)

print("--- Résultat du Test ANOVA ---")
print(f"F-Statistic : {f_stat:.2f}")
print(f"P-value     : {p_value:.5e}")  # .5e affiche en notation scientifique (ex: 1.2e-05)

# 3. Interprétation automatique
if p_value < 0.05:
    print("\n✅ P-value < 0.05 : On REJETTE H0.")
    print("👉 Conclusion : Il y a une différence SIGNIFICATIVE de BMI entre les régions.")
    print("   (Ce n'est pas du hasard, la région impacte vraiment le poids).")
else:
    print("\n❌ P-value > 0.05 : On garde H0.")
    print("👉 Conclusion : Pas de différence significative. Les écarts sont dus au hasard.")

- Nous avons obtenu une F-statistic de 39.50 et une P-value de $1.88 \times 10^{-24}$.
- La P-value étant extrêmement inférieure au seuil de risque de 5% ($0.05$), nous rejetons l'hypothèse nulle ($H_0$).
- Il existe une différence significative de l'IMC moyen entre les régions. 
Les observations visuelles de la Question 1 sont donc confirmées statistiquement : la région Southeast présente bien un IMC moyen significativement différent (plus élevé) des autres régions.

##### Question 4 : Qui est different ? (Post-hoc)

L'analyse des moyennes montre clairement que la région Southeast a l'IMC le plus élevé (33.36) par rapport aux autres régions.
La région Southeast est identifiée comme la zone la plus "à risque" en termes de surpoids. Pour la compagnie d'assurance, cette information est cruciale : cette population présente une probabilité plus élevée de développer des pathologies liées à l'obésité (diabète, maladies cardiovasculaires). Cela pourrait justifier un ajustement des primes ou le lancement de campagnes de prévention ciblées dans cette zone géographique.